# CARNIVAL

CARNIVAL (CAusal Reasoning for Network identification using Integer VALue programming) is a method for the identification of upstream reguatory signalling pathways from downstream gene expression (GEX). Applications of CARNIVAL include the identification of drug’s modes of action and of deregulated processes in diseases (even if the molecular targets remain unknown) by deciphering the alterations of main signalling pathways as well as alternative pathways and off-target effects.

<center style="margin:25px">
    <img src="/_static/carnival-abstract.png" alt="CARNIVAL abstract" style="width: 100%; max-width:800px; margin-bottom:10px;" onerror="this.onerror=null; this.src='https://github.com/saezlab/PerMedCoE_tools_virtual_course_2023/raw/main/assets/carnival_graphical_abstract.png';">
    <br>
    <font size="-1">
        <b>Figure 1:</b> Liu A., Trairatphisan P., Gjerga E. et al. From expression footprints to causal pathways: contextualizing large signaling networks with CARNIVAL npj Systems Biology and Applications volume 5, Article number: 40 (2019) (equal contributions).
    </font>
</center>

The aim of the CARNIVAL pipeline is to identify a subset of interactions from a prior knowledge network that represent potential regulated pathways linking known or potential targets of perturbation towards active transcription factors derived from GEX data. The pipeline includes a number improved functionalities comparing to the original version and consists of the following processes:

- Transcription factors’ (TFs) activities and pathway scores from gene expressions can be inferred with our in-house tools (Dorothea, CollecTRI).
- TFs’ activities and signed directed protein-protein interaction networks with or without the provided target of perturbations and pathway scores are then used to construct an optimization problem with CORNETO.
- CORNETO is used to solve the optimization problem with any of the supported solvers (CPLEX, GUROBI, SCIPY, etc), which identifies the sub-network topology with minimised fitting error and model size.



The original version of CARNIVAL was implemented in R and CPLEX. The new re-implementationo of CARNIVAL in CORNETO support a wide variety of solvers thanks to the support of both CVXPY and PICOS. It also has more flexibility since the problem is symbolically defined, and can be modified through the CORNETO API after creating the CARNIVAL problem. This gives user extra flexibility to modify the problem or to use CORNETO as a building block for other optimization problems. 

In [ ]:
import corneto as cn

cn.info()

## A toy example

In [ ]:
G = cn.Graph.from_sif_tuples(
    [
        ("I1", 1, "N1"),  # I1 activates N1
        ("N1", 1, "M1"),  # N1 activates M1
        ("N1", 1, "M2"),  # N1 activaes M2
        ("I2", -1, "N2"),  # I2 inhibits N2
        ("N2", -1, "M2"),  # N2 inhibits M2
        ("N2", -1, "M1"),  # N2 inhibits M1
    ]
)
G.plot()

In [ ]:
from corneto.methods import runVanillaCarnival

# These are the measurements (e.g. TF activity from Decoupler).
# Positive values correspond to up-regulation and negative values
# with down-regulation. The bigger the absolute value is,
# the bigger the importance is
measurements = {"M1": 1, "M2": 1}

# Perturbations are the upstream nodes were the signal originates on,
# for example, ligands or receptors.
perturbations = {"I1": 1, "I2": 1}

# We run the `standard` carnival problem. This interface is similar
# to the old R function https://saezlab.github.io/CARNIVAL/reference/runVanillaCarnival.html
P, Gf = runVanillaCarnival(perturbations, measurements, G, betaWeight=0.1)

In [ ]:
P.objectives[0].value

In [ ]:
P.objectives[1].value

In [ ]:
# The method for plotting will be simplified in next versions
G.plot(
    custom_edge_attr=cn.pl.edge_style(P, edge_var="edge_values_c0"),
    custom_vertex_attr=cn.pl.vertex_style(P, Gf, vertex_var="vertex_values_c0"),
)

In [ ]:
import pandas as pd

from corneto.methods.carnival import get_result, get_selected_edges

V, E = get_result(P, Gf)
pd.DataFrame(V)

In [ ]:
pd.DataFrame(E)

In [ ]:
G_sol = Gf.edge_subgraph(get_selected_edges(P, Gf))
G_sol.plot()